# Automated Feature Engineering (featuretools)

In [1]:
import featuretools as ft
import pandas as pd
from IPython.display import display
from feature_engine.creation import CyclicalFeatures
from feature_engine.imputation import AddMissingIndicator, ArbitraryNumberImputer, MeanMedianImputer
# from dask.distributed import LocalCluster
from featuretools.primitives import TimeSinceFirst, TimeSinceLast
from tsfresh import extract_features, extract_relevant_features
from tsfresh.feature_extraction import EfficientFCParameters
from woodwork.logical_types import Boolean, BooleanNullable, Categorical, Unknown

from utils import add_calendar_values

In [2]:
pd.set_option('display.float_format', '{:.3f}'.format)
pd.set_option('display.max_columns', None)

## Create EntitySet

In [3]:
es = ft.EntitySet(id='client_data')

In [4]:
"""Clients"""

from utils import read_clients

CLIENTS_PATH = '../data/initial/CLIENTS.csv'
# CLIENTS_PATH = '../data/samples/CLIENTS_SAMPLE.csv'

clients_df = read_clients(CLIENTS_PATH, encode_bool=False)
clients_df = clients_df.astype({'client_id': str})
display(clients_df.info())

es.add_dataframe(
    clients_df,
    dataframe_name='clients',
    index='client_id',
    time_index='communication_month',
    # make_index=True,
    logical_types={
        'client_id': Unknown,
        'target': Boolean,
        'is_train': Boolean,
    }
)
display(es['clients'].ww.schema)
display(es)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36337 entries, 0 to 36336
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   client_id            36337 non-null  object        
 1   target               36337 non-null  bool          
 2   is_train             36337 non-null  bool          
 3   communication_month  36337 non-null  datetime64[ns]
dtypes: bool(2), datetime64[ns](1), object(1)
memory usage: 638.9+ KB


None

,Logical Type,Semantic Tag(s)
Column,,
client_id,Unknown,['index']
target,Boolean,[]
is_train,Boolean,[]
communication_month,Datetime,['time_index']


Entityset: client_data
  DataFrames:
    clients [Rows: 36337, Columns: 4]
  Relationships:
    No relationships

In [5]:
"""Transactions"""

from utils import read_transactions

TRANSACTIONS_PATH = '../data/initial/TRANSACTIONS.csv'
# TRANSACTIONS_PATH = '../data/samples/TRANSACTIONS_SAMPLE.csv'

transactions_df = read_transactions(TRANSACTIONS_PATH, encode_bool=False)
transactions_df = transactions_df.astype({'client_id': str})
transactions_df['tran_date_str'] = transactions_df['tran_date'].dt.date.astype(str)  # used later for feature generation
transactions_df = add_calendar_values(transactions_df, 'tran_date', prefix='tran_date_')
display(transactions_df.info())

es.add_dataframe(
    transactions_df,
    dataframe_name='transactions',
    index='transaction_id',
    time_index='tran_date',
    make_index=True,
    logical_types={
        'client_id': Unknown,
        'cat_c2': Categorical,
        'cat_c3': Categorical,
        'cat_c4': Categorical,
        'tran_date_str': Categorical,
        'fl_c6': Boolean,
        'fl_c7': Boolean,
        'fl_c8': Boolean,
        'fl_c9': Boolean,
        'fl_c10': Boolean,
        'fl_c11': Boolean,
        'fl_c12': Boolean,
        'fl_c13': Boolean,
        'fl_c14': Boolean,
        'fl_c15': Boolean,
    }
)
display(es['transactions'].ww.schema)

es.add_relationship('clients', 'client_id', 'transactions', 'client_id')
display(es)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15654626 entries, 0 to 15654625
Data columns (total 27 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   client_id                   object        
 1   tran_date                   datetime64[ns]
 2   cat_c2                      int32         
 3   cat_c3                      int32         
 4   cat_c4                      int32         
 5   fl_c6                       bool          
 6   fl_c7                       bool          
 7   fl_c8                       bool          
 8   fl_c9                       bool          
 9   fl_c10                      bool          
 10  fl_c11                      bool          
 11  fl_c12                      bool          
 12  fl_c13                      bool          
 13  fl_c14                      bool          
 14  fl_c15                      bool          
 15  float_c16                   float32       
 16  float_c17       

None

,Logical Type,Semantic Tag(s)
Column,,
transaction_id,Integer,['index']
client_id,Unknown,[]
tran_date,Datetime,['time_index']
cat_c2,Categorical,['category']
cat_c3,Categorical,['category']
cat_c4,Categorical,['category']
fl_c6,Boolean,[]
fl_c7,Boolean,[]
fl_c8,Boolean,[]


Entityset: client_data
  DataFrames:
    clients [Rows: 36337, Columns: 4]
    transactions [Rows: 15654626, Columns: 28]
  Relationships:
    transactions.client_id -> clients.client_id

In [6]:
"""App activity"""

from utils import preprocess_app_activity_data, read_app_activity

ACTIVITY_PATH = '../data/initial/APP_ACTIVITY.csv'
# ACTIVITY_PATH = '../data/samples/APP_ACTIVITY_SAMPLE.csv'

activities_df = read_app_activity(ACTIVITY_PATH, encode_bool=False)
activities_df = activities_df.astype({'client_id': str})
activities_df = preprocess_app_activity_data(activities_df)
activities_df['activity_date_str'] = activities_df['activity_date'].dt.date.astype(str)  # used later for feature generation
activities_df = add_calendar_values(activities_df, 'activity_date', prefix='activity_date_')
display(activities_df.info())

es.add_dataframe(
    activities_df,
    dataframe_name='activities',
    index='activity_id',
    time_index='activity_date',
    make_index=True,
    logical_types={
        'client_id': Unknown,
        'cat_c3': Categorical,
        'cat_c4': Categorical,
        'cat_c5': Categorical,
        'cat_c6': Categorical,
        'cat_c9': Categorical,
        'activity_date_str': Categorical,
        'cat_c8': BooleanNullable,
        'cat_c10': BooleanNullable,
    }
)
display(es['activities'].ww.schema)

es.add_relationship('clients', 'client_id', 'activities', 'client_id')
display(es)

<class 'pandas.core.frame.DataFrame'>
Index: 17665187 entries, 0 to 17738591
Data columns (total 19 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   client_id                       object        
 1   device_id                       uint64        
 2   activity_date                   datetime64[ns]
 3   cat_c3                          int32         
 4   cat_c4                          int32         
 5   cat_c5                          int32         
 6   cat_c6                          int32         
 7   cat_c8                          boolean       
 8   cat_c9                          int32         
 9   cat_c10                         boolean       
 10  float_c11                       float32       
 11  float_c12                       float32       
 12  float_c14                       float32       
 13  activity_date_str               object        
 14  activity_date_is_weekend        bool          
 15  a

None

,Logical Type,Semantic Tag(s)
Column,,
activity_id,Integer,['index']
client_id,Unknown,[]
device_id,Integer,['numeric']
activity_date,Datetime,['time_index']
cat_c3,Categorical,['category']
cat_c4,Categorical,['category']
cat_c5,Categorical,['category']
cat_c6,Categorical,['category']
cat_c8,BooleanNullable,[]


Entityset: client_data
  DataFrames:
    clients [Rows: 36337, Columns: 4]
    transactions [Rows: 15654626, Columns: 28]
    activities [Rows: 17665187, Columns: 20]
  Relationships:
    transactions.client_id -> clients.client_id
    activities.client_id -> clients.client_id

In [7]:
"""Communications"""

from utils import read_communications

COMMS_PATH = '../data/initial/COMMUNICATIONS.csv'
# COMMS_PATH = '../data/samples/COMMUNICATIONS_SAMPLE.csv'

comms_df = read_communications(COMMS_PATH)
comms_df = comms_df.astype({'client_id': str})
# comms_df = preprocess_comm_data(comms)
# comms_df = encode_comm_categories(comms)
comms_df['contact_date_str'] = comms_df['contact_date'].dt.date.astype(str)  # used later for feature generation
comms_df = add_calendar_values(comms_df, 'contact_date', prefix='contact_date_')
display(comms_df.info(show_counts=True))

es.add_dataframe(
    comms_df,
    dataframe_name='comms',
    index='comm_id',
    time_index='contact_date',
    make_index=True,
    logical_types={
        'client_id': Unknown,
        'cat_c2': Categorical,
        'cat_c3': Categorical,
        'cat_c4': Categorical,
        'cat_c5': Categorical,
        'contact_date_str': Categorical,
    }
)
display(es['comms'].ww.schema)

es.add_relationship('clients', 'client_id', 'comms', 'client_id')
display(es)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19586922 entries, 0 to 19586921
Data columns (total 12 columns):
 #   Column                         Non-Null Count     Dtype         
---  ------                         --------------     -----         
 0   client_id                      19586922 non-null  object        
 1   contact_date                   19586922 non-null  datetime64[ns]
 2   cat_c2                         19586922 non-null  category      
 3   cat_c3                         19586922 non-null  int32         
 4   cat_c4                         19586922 non-null  int32         
 5   cat_c5                         19586917 non-null  category      
 6   contact_date_str               19586922 non-null  object        
 7   contact_date_is_weekend        19586922 non-null  bool          
 8   contact_date_day_of_week_sin   19586922 non-null  float64       
 9   contact_date_day_of_week_cos   19586922 non-null  float64       
 10  contact_date_day_of_month_sin  19586922 

None

,Logical Type,Semantic Tag(s)
Column,,
comm_id,Integer,['index']
client_id,Unknown,[]
contact_date,Datetime,['time_index']
cat_c2,Categorical,['category']
cat_c3,Categorical,['category']
cat_c4,Categorical,['category']
cat_c5,Categorical,['category']
contact_date_str,Categorical,['category']
contact_date_is_weekend,Boolean,[]


Entityset: client_data
  DataFrames:
    clients [Rows: 36337, Columns: 4]
    transactions [Rows: 15654626, Columns: 28]
    activities [Rows: 17665187, Columns: 20]
    comms [Rows: 19586922, Columns: 13]
  Relationships:
    transactions.client_id -> clients.client_id
    activities.client_id -> clients.client_id
    comms.client_id -> clients.client_id

## Deep Feature Synthesis

In [8]:
MIN_DATE_STR = '2024-12-01 00:00:00'
MAX_DATE_STR = '2025-09-01 00:00:00'

agg_primitives = [
    'count',
    'first', 'last', TimeSinceFirst(unit='days'), TimeSinceLast(unit='days'),  # for date
    'mean', 'std', 'skew', 'kurtosis', 'min', 'max', 'sum',   # for numeric
    'num_unique', 'mode',  # for categorical
    'percent_true',  # for boolean
]

primitive_options = {}
for agg in agg_primitives:
    primitive_options[agg]= {'include_columns': {}}
    for df_name in ('transactions', 'activities', 'comms'):
         if agg not in ('count',) and isinstance(agg, str):
             primitive_options[agg]['include_columns'][df_name] = []

In [9]:
"""Transactions"""

df_name = 'transactions'

# Date
for prim in ['first', 'last']:
    primitive_options[prim]['include_columns'][df_name] += ['tran_date']
primitive_options['mean']['include_columns'][df_name] += [
    'tran_date_day_of_week_sin', 'tran_date_day_of_week_cos',
    'tran_date_day_of_month_sin', 'tran_date_day_of_month_cos',
]

# Numeric
num_cols = ['float_c16', 'float_c17', 'float_c18', 'int_c19', 'float_c20', 'float_c21']
for prim in ['mean', 'std', 'skew', 'kurtosis', 'min', 'max', 'sum']:
    primitive_options[prim]['include_columns'][df_name] += num_cols
    # primitive_options[prim] = {'include_columns': {df_name: num_cols}}

# Categorical
cat_cols = list(es[df_name].ww.select(Categorical).columns)
for prim in ['num_unique', 'mode']:
    # primitive_options['mode'] = {'include_columns': {df_name: cat_cols}}
    primitive_options[prim]['include_columns'][df_name] += cat_cols
primitive_options['mode']['include_columns'][df_name].remove('tran_date_str')  # delete redundant primitive

# Boolean
bool_cols = list(es[df_name].ww.select([Boolean, BooleanNullable]).columns)
# primitive_options['percent_true'] = {'include_columns': {df_name: bool_cols}}
primitive_options['percent_true']['include_columns'][df_name] += bool_cols


# Interesting values
interesting_values = {
    'cat_c2': [4, 14, 15],
    'cat_c3': [209, 303, 305, 314],
    'int_c19': [-1, 1],
    'fl_c12': [True, False],
    'fl_c13': [True, False],
    'fl_c14': [True, False],
    'tran_date_is_weekend': [True, False],
}
es.add_interesting_values(dataframe_name=df_name, values=interesting_values)

In [10]:
"""Activities"""

df_name = 'activities'

# Date
for prim in ['first', 'last']:
    primitive_options[prim]['include_columns'][df_name] += ['activity_date']
primitive_options['mean']['include_columns'][df_name] += [
    'activity_date_day_of_week_sin', 'activity_date_day_of_week_cos',
    'activity_date_day_of_month_sin', 'activity_date_day_of_month_cos',
]

# Numeric
num_cols = ['float_c11', 'float_c12', 'float_c13', 'float_c14', 'float_c15', 'float_c16', 'float_c17']
for prim in ['mean', 'std', 'skew', 'kurtosis', 'min', 'max', 'sum']:
    primitive_options[prim]['include_columns'][df_name] += num_cols

# Categorical
cat_cols = list(es[df_name].ww.select(Categorical).columns)
for prim in ['num_unique', 'mode']:
    primitive_options[prim]['include_columns'][df_name] += cat_cols
primitive_options['mode']['include_columns'][df_name].remove('activity_date_str')  # delete redundant primitive

# Boolean
bool_cols = list(es[df_name].ww.select([Boolean, BooleanNullable]).columns)
primitive_options['percent_true']['include_columns'][df_name] += bool_cols


# Interesting values
interesting_values = {
    'cat_c4': [1, 2],
    'cat_c6': [1, 2, 3],
    'cat_c9': [1, 2],
    'cat_c8': [True, False],
    'cat_c10': [True, False],
    'activity_date_is_weekend': [True, False],
}
es.add_interesting_values(dataframe_name=df_name, values=interesting_values)

In [11]:
"""Communications"""

df_name = 'comms'

# Date
for prim in ['first', 'last']:
    primitive_options[prim]['include_columns'][df_name] += ['contact_date']
primitive_options['mean']['include_columns'][df_name] += [
    'contact_date_day_of_week_sin', 'contact_date_day_of_week_cos',
    'contact_date_day_of_month_sin', 'contact_date_day_of_month_cos',
]

# Categorical
cat_cols = list(es[df_name].ww.select(Categorical).columns)
for prim in ['num_unique', 'mode']:
    primitive_options[prim]['include_columns'][df_name] += cat_cols
primitive_options['mode']['include_columns'][df_name].remove('contact_date_str')  # delete redundant primitive


# Boolean
bool_cols = list(es[df_name].ww.select([Boolean, BooleanNullable]).columns)
primitive_options['percent_true']['include_columns'][df_name] += bool_cols

# Interesting values
interesting_values = {
    'cat_c2': ['S3564', 'S3565', 'S3677', 'S3769'],
    'cat_c3': [3, 4, 7],
    'cat_c4': [1, 2],
    'cat_c5': ['4', '7'],
    'contact_date_is_weekend': [True, False],
}
es.add_interesting_values(dataframe_name=df_name, values=interesting_values)

In [12]:
# cluster = LocalCluster()
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name='clients',
    # ignore_dataframes=['transactions', 'comms'],
    # cutoff_time=transactions_df['tran_date'].max(),
    cutoff_time=MAX_DATE_STR,
    agg_primitives=agg_primitives,
    trans_primitives=['day', 'month', 'weekday', 'is_weekend'],
    where_primitives=['mean', 'sum', 'count'],
    primitive_options=primitive_options,
    max_depth=2,
    verbose=True,
    features_only=False,
    n_jobs=1,
    # chunk_size=.05,
    # dask_kwargs={'cluster': cluster.scheduler.address},
    return_types='all',
)
feature_matrix_enc, features_enc = ft.encode_features(
    feature_matrix,
    feature_defs,
    top_n={
        # Transactions
        'MODE(transactions.cat_c2)': 10,
        'MODE(transactions.cat_c3)': 10,
        'MODE(transactions.cat_c4)': 5,

        # Activities
        'MODE(activities.cat_c3)': 3,
        'MODE(activities.cat_c4)': 2,
        'MODE(activities.cat_c5)': 1,
        'MODE(activities.cat_c6)': 7,
        'MODE(activities.cat_c9)': 2,

        # Communications
        'MODE(comms.cat_c2)': 10,
        'MODE(comms.cat_c3)': 3,
        'MODE(comms.cat_c4)': 3,
        'MODE(comms.cat_c5)': 3,
    },
    to_encode=[
        'MODE(transactions.cat_c2)', 'MODE(transactions.cat_c3)', 'MODE(transactions.cat_c4)',  # transactions
        'MODE(activities.cat_c3)', 'MODE(activities.cat_c4)', 'MODE(activities.cat_c5)', 'MODE(activities.cat_c6)', 'MODE(activities.cat_c9)',  # activities
        'MODE(comms.cat_c2)', 'MODE(comms.cat_c3)', 'MODE(comms.cat_c4)', 'MODE(comms.cat_c5)',  # communications
    ],
)
ft.save_features(features_enc, '../data/features/feature_definitions_v3.json')
feature_matrix_enc.to_csv('../data/features/features_auto_v3_raw.csv', index=False)

display(features_enc)
display(feature_matrix_enc.head())
display(feature_matrix_enc.info(verbose=True, show_counts=True))
display(feature_matrix_enc.describe())

C:\Users\dan\PycharmProjects\ds\ff_internship\.venv\Lib\site-packages\featuretools\primitives\options_utils.py:39: UserWarning: Column 'float_c13' not in dataframe 'activities'
  warnings.warn(
C:\Users\dan\PycharmProjects\ds\ff_internship\.venv\Lib\site-packages\featuretools\primitives\options_utils.py:39: UserWarning: Column 'float_c15' not in dataframe 'activities'
  warnings.warn(
C:\Users\dan\PycharmProjects\ds\ff_internship\.venv\Lib\site-packages\featuretools\primitives\options_utils.py:39: UserWarning: Column 'float_c16' not in dataframe 'activities'
  warnings.warn(
C:\Users\dan\PycharmProjects\ds\ff_internship\.venv\Lib\site-packages\featuretools\primitives\options_utils.py:39: UserWarning: Column 'float_c17' not in dataframe 'activities'
  warnings.warn(


Built 680 features
Elapsed: 03:08 | Progress:  18%|█▊        

C:\Users\dan\PycharmProjects\ds\ff_internship\.venv\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:763: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ).apply(wrap)
C:\Users\dan\PycharmProjects\ds\ff_internship\.venv\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function std at 0x000001C5340EBBA0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  ).agg(to_agg)
C:\Users\dan\PycharmProjects\ds\ff_internship\.venv\Lib\site-packages\featuretools\computational_backends\feature_se

Elapsed: 07:37 | Progress:  24%|██▍       

C:\Users\dan\PycharmProjects\ds\ff_internship\.venv\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function sum at 0x000001C5340EA660> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ).agg(to_agg)
C:\Users\dan\PycharmProjects\ds\ff_internship\.venv\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function mean at 0x000001C5340EBA60> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ).agg(to_agg)


...

[<Feature: target>,
 <Feature: is_train>,
 <Feature: communication_month>,
 <Feature: COUNT(transactions)>,
 <Feature: FIRST(transactions.tran_date)>,
 <Feature: KURTOSIS(transactions.float_c16)>,
 <Feature: KURTOSIS(transactions.float_c17)>,
 <Feature: KURTOSIS(transactions.float_c18)>,
 <Feature: KURTOSIS(transactions.float_c20)>,
 <Feature: KURTOSIS(transactions.float_c21)>,
 <Feature: KURTOSIS(transactions.int_c19)>,
 <Feature: LAST(transactions.tran_date)>,
 <Feature: MAX(transactions.float_c16)>,
 <Feature: MAX(transactions.float_c17)>,
 <Feature: MAX(transactions.float_c18)>,
 <Feature: MAX(transactions.float_c20)>,
 <Feature: MAX(transactions.float_c21)>,
 <Feature: MAX(transactions.int_c19)>,
 <Feature: MEAN(transactions.float_c16)>,
 <Feature: MEAN(transactions.float_c17)>,
 <Feature: MEAN(transactions.float_c18)>,
 <Feature: MEAN(transactions.float_c20)>,
 <Feature: MEAN(transactions.float_c21)>,
 <Feature: MEAN(transactions.int_c19)>,
 <Feature: MEAN(transactions.tran_date_

,target,is_train,communication_month,COUNT(transactions),FIRST(transactions.tran_date),KURTOSIS(transactions.float_c16),KURTOSIS(transactions.float_c17),KURTOSIS(transactions.float_c18),KURTOSIS(transactions.float_c20),KURTOSIS(transactions.float_c21),KURTOSIS(transactions.int_c19),LAST(transactions.tran_date),MAX(transactions.float_c16),MAX(transactions.float_c17),MAX(transactions.float_c18),MAX(transactions.float_c20),MAX(transactions.float_c21),MAX(transactions.int_c19),MEAN(transactions.float_c16),MEAN(transactions.float_c17),MEAN(transactions.float_c18),MEAN(transactions.float_c20),MEAN(transactions.float_c21),MEAN(transactions.int_c19),MEAN(transactions.tran_date_day_of_month_cos),MEAN(transactions.tran_date_day_of_month_sin),MEAN(transactions.tran_date_day_of_week_cos),MEAN(transactions.tran_date_day_of_week_sin),MIN(transactions.float_c16),MIN(transactions.float_c17),MIN(transactions.float_c18),MIN(transactions.float_c20),MIN(transactions.float_c21),MIN(transactions.int_c19),NUM_UNIQUE(transactions.cat_c2),NUM_UNIQUE(transactions.cat_c3),NUM_UNIQUE(transactions.cat_c4),NUM_UNIQUE(transactions.tran_date_str),PERCENT_TRUE(transactions.fl_c10),PERCENT_TRUE(transactions.fl_c11),PERCENT_TRUE(transactions.fl_c12),PERCENT_TRUE(transactions.fl_c13),PERCENT_TRUE(transactions.fl_c14),PERCENT_TRUE(transactions.fl_c15),PERCENT_TRUE(transactions.fl_c6),PERCENT_TRUE(transactions.fl_c7),PERCENT_TRUE(transactions.fl_c8),PERCENT_TRUE(transactions.fl_c9),PERCENT_TRUE(transactions.tran_date_is_weekend),SKEW(transactions.float_c16),SKEW(transactions.float_c17),SKEW(transactions.float_c18),SKEW(transactions.float_c20),SKEW(transactions.float_c21),SKEW(transactions.int_c19),STD(transactions.float_c16),STD(transactions.float_c17),STD(transactions.float_c18),STD(transactions.float_c20),STD(transactions.float_c21),STD(transactions.int_c19),SUM(transactions.float_c16),SUM(transactions.float_c17),SUM(transactions.float_c18),SUM(transactions.float_c20),SUM(transactions.float_c21),SUM(transactions.int_c19),"TIME_SINCE_FIRST(transactions.tran_date, unit=days)","TIME_SINCE_LAST(transactions.tran_date, unit=days)",COUNT(activities),FIRST(activities.activity_date),KURTOSIS(activities.float_c11),KURTOSIS(activities.float_c12),KURTOSIS(activities.float_c14),LAST(activities.activity_date),MAX(activities.float_c11),MAX(activities.float_c12),MAX(activities.float_c14),MEAN(activities.activity_date_day_of_month_cos),MEAN(activities.activity_date_day_of_month_sin),MEAN(activities.activity_date_day_of_week_cos),MEAN(activities.activity_date_day_of_week_sin),MEAN(activities.float_c11),MEAN(activities.float_c12),MEAN(activities.float_c14),MIN(activities.float_c11),MIN(activities.float_c12),MIN(activities.float_c14),NUM_UNIQUE(activities.activity_date_str),NUM_UNIQUE(activities.cat_c3),NUM_UNIQUE(activities.cat_c4),NUM_UNIQUE(activities.cat_c5),NUM_UNIQUE(activities.cat_c6),NUM_UNIQUE(activities.cat_c9),PERCENT_TRUE(activities.activity_date_is_weekend),PERCENT_TRUE(activities.cat_c10),PERCENT_TRUE(activities.cat_c8),SKEW(activities.float_c11),SKEW(activities.float_c12),SKEW(activities.float_c14),STD(activities.float_c11),STD(activities.float_c12),STD(activities.float_c14),SUM(activities.float_c11),SUM(activities.float_c12),SUM(activities.float_c14),"TIME_SINCE_FIRST(activities.activity_date, unit=days)","TIME_SINCE_LAST(activities.activity_date, unit=days)",COUNT(comms),FIRST(comms.contact_date),LAST(comms.contact_date),MEAN(comms.contact_date_day_of_month_cos),MEAN(comms.contact_date_day_of_month_sin),MEAN(comms.contact_date_day_of_week_cos),MEAN(comms.contact_date_day_of_week_sin),NUM_UNIQUE(comms.cat_c2),NUM_UNIQUE(comms.cat_c3),NUM_UNIQUE(comms.cat_c4),NUM_UNIQUE(comms.cat_c5),NUM_UNIQUE(comms.contact_date_str),PERCENT_TRUE(comms.contact_date_is_weekend),"TIME_SINCE_FIRST(comms.contact_date, unit=days)","TIME_SINCE_LAST(comms.contact_date, unit=days)",DAY(communication_month),IS_WEEKEND(communication_month),MONTH(communication_month),WEEKDAY(communication_mon

<class 'pandas.core.frame.DataFrame'>
Index: 36337 entries, 10000196835799192770 to 9997098291014878881
Data columns (total 739 columns):
 #    Column                                                                                  Non-Null Count  Dtype         
---   ------                                                                                  --------------  -----         
 0    target                                                                                  36337 non-null  bool          
 1    is_train                                                                                36337 non-null  bool          
 2    communication_month                                                                     36337 non-null  datetime64[ns]
 3    COUNT(transactions)                                                                     36337 non-null  Int64         
 4    FIRST(transactions.tran_date)                                                           35070 non-null  da

None

,communication_month,COUNT(transactions),FIRST(transactions.tran_date),KURTOSIS(transactions.float_c16),KURTOSIS(transactions.float_c17),KURTOSIS(transactions.float_c18),KURTOSIS(transactions.float_c20),KURTOSIS(transactions.float_c21),KURTOSIS(transactions.int_c19),LAST(transactions.tran_date),MAX(transactions.float_c16),MAX(transactions.float_c17),MAX(transactions.float_c18),MAX(transactions.float_c20),MAX(transactions.float_c21),MAX(transactions.int_c19),MEAN(transactions.float_c16),MEAN(transactions.float_c17),MEAN(transactions.float_c18),MEAN(transactions.float_c20),MEAN(transactions.float_c21),MEAN(transactions.int_c19),MEAN(transactions.tran_date_day_of_month_cos),MEAN(transactions.tran_date_day_of_month_sin),MEAN(transactions.tran_date_day_of_week_cos),MEAN(transactions.tran_date_day_of_week_sin),MIN(transactions.float_c16),MIN(transactions.float_c17),MIN(transactions.float_c18),MIN(transactions.float_c20),MIN(transactions.float_c21),MIN(transactions.int_c19),NUM_UNIQUE(transactions.cat_c2),NUM_UNIQUE(transactions.cat_c3),NUM_UNIQUE(transactions.cat_c4),NUM_UNIQUE(transactions.tran_date_str),PERCENT_TRUE(transactions.fl_c10),PERCENT_TRUE(transactions.fl_c11),PERCENT_TRUE(transactions.fl_c12),PERCENT_TRUE(transactions.fl_c13),PERCENT_TRUE(transactions.fl_c14),PERCENT_TRUE(transactions.fl_c15),PERCENT_TRUE(transactions.fl_c6),PERCENT_TRUE(transactions.fl_c7),PERCENT_TRUE(transactions.fl_c8),PERCENT_TRUE(transactions.fl_c9),PERCENT_TRUE(transactions.tran_date_is_weekend),SKEW(transactions.float_c16),SKEW(transactions.float_c17),SKEW(transactions.float_c18),SKEW(transactions.float_c20),SKEW(transactions.float_c21),SKEW(transactions.int_c19),STD(transactions.float_c16),STD(transactions.float_c17),STD(transactions.float_c18),STD(transactions.float_c20),STD(transactions.float_c21),STD(transactions.int_c19),SUM(transactions.float_c16),SUM(transactions.float_c17),SUM(transactions.float_c18),SUM(transactions.float_c20),SUM(transactions.float_c21),SUM(transactions.int_c19),"TIME_SINCE_FIRST(transactions.tran_date, unit=days)","TIME_SINCE_LAST(transactions.tran_date, unit=days)",COUNT(activities),FIRST(activities.activity_date),KURTOSIS(activities.float_c11),KURTOSIS(activities.float_c12),KURTOSIS(activities.float_c14),LAST(activities.activity_date),MAX(activities.float_c11),MAX(activities.float_c12),MAX(activities.float_c14),MEAN(activities.activity_date_day_of_month_cos),MEAN(activities.activity_date_day_of_month_sin),MEAN(activities.activity_date_day_of_week_cos),MEAN(activities.activity_date_day_of_week_sin),MEAN(activities.float_c11),MEAN(activities.float_c12),MEAN(activities.float_c14),MIN(activities.float_c11),MIN(activities.float_c12),MIN(activities.float_c14),NUM_UNIQUE(activities.activity_date_str),NUM_UNIQUE(activities.cat_c3),NUM_UNIQUE(activities.cat_c4),NUM_UNIQUE(activities.cat_c5),NUM_UNIQUE(activities.cat_c6),NUM_UNIQUE(activities.cat_c9),PERCENT_TRUE(activities.activity_date_is_weekend),PERCENT_TRUE(activities.cat_c10),PERCENT_TRUE(activities.cat_c8),SKEW(activities.float_c11),SKEW(activities.float_c12),SKEW(activities.float_c14),STD(activities.float_c11),STD(activities.float_c12),STD(activities.float_c14),SUM(activities.float_c11),SUM(activities.float_c12),SUM(activities.float_c14),"TIME_SINCE_FIRST(activities.activity_date, unit=days)","TIME_SINCE_LAST(activities.activity_date, unit=days)",COUNT(comms),FIRST(comms.contact_date),LAST(comms.contact_date),MEAN(comms.contact_date_day_of_month_cos),MEAN(comms.contact_date_day_of_month_sin),MEAN(comms.contact_date_day_of_week_cos),MEAN(comms.contact_date_day_of_week_sin),NUM_UNIQUE(comms.cat_c2),NUM_UNIQUE(comms.cat_c3),NUM_UNIQUE(comms.cat_c4),NUM_UNIQUE(comms.cat_c5),NUM_UNIQUE(comms.contact_date_str),PERCENT_TRUE(comms.contact_date_is_weekend),"TIME_SINCE_FIRST(comms.contact_date, unit=days)","TIME_SINCE_LAST(comms.contact_date, unit=days)",COUNT(transactions WHERE cat_c2 = 4),COUNT(transactions WHERE cat_c3 = 314),COUNT(transactions WHERE fl_c14 = True),COUNT(tra

## Add more time-series features

In [13]:
def prep_time_gaps(df, date_col, prefix):
    df = df.sort_values(by=['client_id', date_col])

    gap_col_name = f'{prefix}_gap_days'
    df[gap_col_name] = df.groupby('client_id')[date_col].diff().dt.total_seconds() / (3600 * 24)
    df[gap_col_name] = df[gap_col_name].fillna(0)

    return df[['client_id', date_col, gap_col_name]]

df_transactions_ts = prep_time_gaps(transactions_df, 'tran_date', 'tran_date')
df_transactions_ts['float_c18_diff'] = transactions_df.groupby('client_id')['float_c18'].diff().fillna(0)  # difference between transactions amount
df_activities_ts = prep_time_gaps(activities_df, 'activity_date', 'activity_date')
df_comms_ts  = prep_time_gaps(comms_df, 'contact_date', 'contact_date')

display(df_transactions_ts.head(10))

,client_id,tran_date,tran_date_gap_days,float_c18_diff
461179,10000196835799192770,2024-12-01,0.000,0.000
461290,10000196835799192770,2024-12-01,0.000,31.620
461139,10000196835799192770,2024-12-03,2.000,-28.340
461189,10000196835799192770,2024-12-03,0.000,-0.690
461143,10000196835799192770,2024-12-05,2.000,-1.400
461195,10000196835799192770,2024-12-05,0.000,18.690
461117,10000196835799192770,2024-12-07,2.000,-17.310
461146,10000196835799192770,2024-12-07,0.000,30.510
461201,10000196835799192770,2024-12-07,0.000,86.800
461202,10000196835799192770,2024-12-07,0.000,-40.000


In [14]:
datasets = {
    'transactions': (df_transactions_ts, 'tran_date', ['tran_date_gap_days', 'float_c18_diff']),
    'activities': (df_activities_ts, 'activity_date', ['activity_date_gap_days']),
    'communications': (df_comms_ts, 'contact_date', ['contact_date_gap_days'])
}
all_features = []
for name, (df, date_col, val_cols) in datasets.items():
    print(f'Processing {name}...')

    for val_col in val_cols:
        fc_parameters = {
            'median': None,
            'mean': None,
            'maximum': None,
            'minimum': None,
            'standard_deviation': None,
            'skewness': None,
            'kurtosis': None,
            'quantile': [
                {'q': 0.1},
                {'q': 0.25},
                {'q': 0.75},
                {'q': 0.9}
            ],
            'linear_trend': [{'attr': 'slope'}],
            'approximate_entropy': [{'m': 2, 'r': 0.25}],
            'ratio_beyond_r_sigma': [{'r': 2}],
            'autocorrelation': [{'lag': 1}],
            'fft_aggregated': [{'aggtype': 'centroid'}, {'aggtype': 'variance'}],
            'longest_strike_above_mean': None,
            'longest_strike_below_mean': None,
            'count_above_mean': None,
        }
        features = extract_features(
            timeseries_container=df,
            column_id='client_id',
            column_sort=date_col,
            column_value=val_col,
            default_fc_parameters=fc_parameters,
            n_jobs=14,
        )
        features = features.reset_index(names='client_id')
        all_features.append(features)

final_features = feature_matrix_enc
for features in all_features:
    final_features = final_features.merge(features, on='client_id', how='left')

display(final_features.info(verbose=True, show_counts=True))

Processing transactions...


Feature Extraction: 100%|██████████| 70/70 [07:17<00:00,  6.24s/it] 


Processing activities...


Feature Extraction: 100%|██████████| 70/70 [08:33<00:00,  7.33s/it] 


Processing communications...


Feature Extraction: 100%|██████████| 70/70 [09:45<00:00,  8.37s/it]  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36337 entries, 0 to 36336
Data columns (total 820 columns):
 #    Column                                                                                  Non-Null Count  Dtype         
---   ------                                                                                  --------------  -----         
 0    client_id                                                                               36337 non-null  string        
 1    target                                                                                  36337 non-null  bool          
 2    is_train                                                                                36337 non-null  bool          
 3    communication_month                                                                     36337 non-null  datetime64[ns]
 4    COUNT(transactions)                                                                     36337 non-null  Int64         
 5    FIRST(tra

None

## Postprocessing

In [15]:
df = final_features
df = df.reset_index()
# df = df.drop(columns=['client_id'])
df = df.dropna(subset=[
    # Transactions
    'NUM_UNIQUE(transactions.tran_date_str)', 'MEAN(transactions.float_c16 WHERE int_c19 = -1)', 'MEAN(transactions.float_c16 WHERE int_c19 = 1)',

    # Activities
    'NUM_UNIQUE(activities.activity_date_str)', 'STD(activities.float_c11)',

    # Communications
    'NUM_UNIQUE(comms.contact_date_str)',
])

# Convert dates to days from initial point
for date_col in df.select_dtypes(include=['datetime64[ns]']).columns:
    df[date_col + '_days'] = (df[date_col] - pd.to_datetime(MIN_DATE_STR)).dt.days
    df = df.drop(columns=[date_col])

# Encode cyclical features (e.g. day of week, month)
time_cols = ['DAY(communication_month)', 'MONTH(communication_month)', 'WEEKDAY(communication_month)']
df = df.astype({col: 'int8' for col in time_cols})
cyclical = CyclicalFeatures(variables=time_cols, drop_original=True)
df = cyclical.fit_transform(df)

# Fill  missing data
cols_to_fill_with_mean = [
    c for c in df.columns
    if (' WHERE ' in c and c.startswith('MEAN(')
        or c.startswith('SKEW(')
        or c.endswith('__skewness')
        or c.endswith('__kurtosis')
        or '__autocorrelation' in c
        or '__fft_aggregated__aggtype' in c)
]
ami = AddMissingIndicator(variables=cols_to_fill_with_mean)
df = ami.fit_transform(df)
mmi = MeanMedianImputer(imputation_method='mean', variables=cols_to_fill_with_mean)
df = mmi.fit_transform(df)

# Convert boolean values to 0 and 1
df = df.astype({col: 'int8' for col in df.select_dtypes(include=['bool']).columns})
df = df.astype({col: 'Int8' for col in df.select_dtypes(include=['boolean']).columns})

df.to_csv('../data/features/features_auto_v3.csv', index=False)
display(df.info(verbose=True, show_counts=True))
display(df.describe())


C:\Users\dan\PycharmProjects\ds\ff_internship\.venv\Lib\site-packages\feature_engine\imputation\missing_indicator.py:165: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[indicator_names] = X[self.variables_].isna().astype(int)
C:\Users\dan\PycharmProjects\ds\ff_internship\.venv\Lib\site-packages\feature_engine\imputation\missing_indicator.py:165: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[indicator_names] = X[self.variables_].isna().astype(int)
C:\Users\dan\PycharmProjects\ds\ff_internship\.venv\Lib\site-packages\feature_en

<class 'pandas.core.frame.DataFrame'>
Index: 34837 entries, 0 to 36336
Data columns (total 1098 columns):
 #     Column                                                                                     Non-Null Count  Dtype   
---    ------                                                                                     --------------  -----   
 0     index                                                                                      34837 non-null  int64   
 1     client_id                                                                                  34837 non-null  string  
 2     target                                                                                     34837 non-null  int8    
 3     is_train                                                                                   34837 non-null  int8    
 4     COUNT(transactions)                                                                        34837 non-null  Int64   
 5     KURTOSIS(transactions.floa

None

index    target  is_train  COUNT(transactions)  \
count 34837.000 34837.000 34837.000            34837.000   
mean  18126.683     0.245     0.793              449.323   
std   10500.226     0.430     0.405              474.248   
min       0.000     0.000     0.000                2.000   
25%    9029.000     0.000     1.000              124.000   
50%   18098.000     0.000     1.000              311.000   
75%   27227.000     0.000     1.000              618.000   
max   36336.000     1.000     1.000             8721.000   

       KURTOSIS(transactions.float_c16)  KURTOSIS(transactions.float_c17)  \
count                         34837.000                         34837.000   
mean                             79.004                           132.896   
std                             147.101                           225.436   
min                              -2.000                            -2.000   
25%                               0.000                             0.000   
50%                              27.937                            34.824   
75%                              93.501                           177.363   
max                            3572.596                          2741.303   

       KURTOSIS(transactions.float_c18)  KURTOSIS(transactions.float_c20)  \
count                         34837.000                         34837.000   
mean                             42.252                            16.109   
std                              58.002                           112.742   
min                              -2.000                            -2.000   
25%                              11.223                             0.000   
50%                              25.631                             0.000   
75%                              51.324                             0.000   
max                            1350.809                          3767.000   

       KURTOSIS(transactions.float_c21)  KURTOSIS(transactions.int_c19)  \
count                         34837.000                       34837.000   
mean                             43.584                          -0.722   
std                              62.475                           2.658   
min                              -2.000                          -2.000   
25%                              11.208                          -1.809   
50%                              25.924                          -1.332   
75%                              52.299                          -0.397   
max                            1588.049                         259.670   

       MAX(transactions.float_c16)  MAX(transactions.float_c17)  \
count                    34837.000                    34837.000   
mean                         1.626                        4.584   
std                          4.876                       19.238   
min                          0.000                        0.000   
25%                          0.000                        0.000   
50%                          0.480                        1.000   
75%                          1.670                        4.030   
max                        413.130                     2294.720   

       MAX(transactions.float_c18)  MAX(transactions.float_c20)  \
count                    34837.000                    34837.000   
mean                      1446.551                       80.504   
std                       1648.496                      486.876   
min                          0.010                        0.000   
25%                        510.000                        0.000   
50%                       1000.000                        0.000   
75%                       1963.430                        0.000   
max                      55600.000                    13500.000   

       MAX(transactions.float_c21)  MAX(transactions.int_c19)  \
count                    34837.000                  34837.000   
mean                      1418.000                      1.000   
st